In [10]:
import os
from dotenv import load_dotenv
from autogen import ConversableAgent
from autogen.coding import DockerCommandLineCodeExecutor

load_dotenv()  # take environment variables from .env.

llm_config={"config_list": [{
    "model": "gpt-4-turbo",
    "cache": None,
    "temperature": 0.9, 
    "api_key": os.environ.get("OPENAI_API_KEY")}]}


# Create a temporary directory to store the code files.
temp_dir = './tmp'

docker_container_name = 'autogen'

docker_executor = DockerCommandLineCodeExecutor(
    image="python:3.12-slim",  # Execute code using the given docker image name.
    container_name=docker_container_name,  # Name of the Docker container.
    timeout=180,  # Timeout for each code execution in seconds - 3 minutes
    work_dir=temp_dir,  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration that uses docker.
code_executor_agent_using_docker = ConversableAgent(
    "code_executor_agent_docker",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": docker_executor},  # Use the docker command line code executor.
    human_input_mode="NEVER",  # Change to ALWAYS to take human input for this agent for safety.
)

message_with_code_block = """This is a message with code block.
The code block is below:
```shell
pip install matplotlib numpy
```
This is the end of the message.
"""

reply = code_executor_agent_using_docker.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
print(reply)


message_with_code_block = """This is a message with code block.
The code block is below:
```python
import numpy as np
import matplotlib.pyplot as plt
x = range(100)
y = np.random.randint(0, 100, 100)
plt.plot(x, y)
plt.savefig('line.png')
print('Scatter plot saved to line.png')
```
This is the end of the message.
"""

reply = code_executor_agent_using_docker.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
print(reply)


>>>>>>>> EXECUTING CODE BLOCK (inferred language is shell)...
exitcode: 0 (execution succeeded)
Code output: 

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 0 (execution succeeded)
Code output: Scatter plot saved to line.png

